In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
#from langchain.cache import InMemoryCache, SQLiteCache
from langchain.schema import BaseOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate

#set_llm_cache(SQLiteCache("cache.db"))

chat = ChatOpenAI(
    temperature=0.1
)

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

# 리스트를 쉼표로 구분된 문자열로 변환하는 함수
def convert_list_to_string(activity_list):
    return ', '.join(activity_list)


template = ChatPromptTemplate.from_messages(
    [
        ("system", 
        """
        비행기를 타고 해외 여행을 갈때 필요한 짐을 알려주세요.
        답변은 아래 요구사항을 참고하세요.
        1. 아래 표기된 각 항목별로 {items} 개 이상의 짐을 말해주세요.
        2. 한국어로만 답변해 주세요.
        3. 데이터를 가공하기 쉽도록 Dictionaly 형태로 만들어 주세요. 예를들어 Key 값은 필수 항목이 될거고 Value 값은 List 형태가 되어야 합니다.
        4. 중복된 짐은 제거해주세요
        - 필수 항목: 해외 여행을 갈때 꼭 챙겨가야할 것. 예시=여권 및 비자, 항공권, 숙박 예약 확인서, 국제 운전 면허증, 여행자 보험, 현금 및 신용카드/체크카드
        - 옷: 여행 목적지의 날씨에 따라 옷을 추천 해줘야 함. 예시=긴 양말, 속옷, 재킷, 후드티
        - 세면 도구: 예시=콘택트 렌즈, 렌즈 세척액, 칫솔, 치약, 빗, 치실, 면도기, 쉐이빙젤, 면봉
        - 활동: 여행 중 즐길 거리에 대한 짐 예시=수영복, 선글라스, 자외선 차단제, 등산 스틱
        - 날씨: 여행 일정에 맞게 날씨 정보를 상세하게 말해준다. 예시= 목적지의 1월 1일 부터 1월 5일까지의 날씨는 평균적으로 최저기온 -2도, 최고기온 5도가 될 예정입니다.
        """),
        # 여기서부터 실제 사용자의 입력을 처리합니다.
        (
            "human",
            """
            저는 {gender}이고, {destination}으로 해외 여행을 갈 예정입니다. \
            {start_month}월 {start_day}일에 출발해서 {days}일 동안 머물 예정입니다. \
            여행 중에는 {activities}를 즐길 계획입니다. \
            위 사항을 참고하여 가능한 자세한 체크리스트 항목을 만들어주세요.
            """
        ),
    ]
)

activities_list = ["수영", "등산"]

input_data = {
    "items": "3",
    "gender": "man",
    "destination": "Seoul",
    "start_month": "1",
    "start_day": "1",
    "days": "5",
    "activities": convert_list_to_string(activities_list),
}

chain = template | chat #| CommaOutputParser()
# Invoke the chain with the input
output = chain.invoke(input_data)

# Print the output
print(output)



content='{\n    "필수 항목": ["여권 및 비자", "항공권", "숙박 예약 확인서", "국제 운전 면허증", "여행자 보험", "현금", "신용카드/체크카드"],\n    "옷": ["겨울 외투", "니트", "기모의류", "두꺼운 양말", "장갑"],\n    "세면 도구": ["칫솔", "치약", "면도기", "쉐이빙젤", "면봉"],\n    "활동": ["수영복", "수영모자", "비치타올", "등산화", "등산 지팡이"],\n    "날씨": "서울의 1월 1일부터 5일까지의 날씨는 평균 최저기온이 -5도, 최고기온이 2도로 추워요."\n}'
